## Monarch + TorchTitan on SLURM
This example notebook demonstrates how you can easily run and iterate on a distributed training job with Monarch and TorchTitan.

#### Prerequisites
Please make sure your environment is setup for this notebook:
1. Install Monarch nightly: https://github.com/meta-pytorch/monarch/blob/main/scripts/install_nightly.py
2. Install Titan nightly: https://github.com/pytorch/torchtitan?tab=readme-ov-file#nightly-builds
3. Ensure you have a valid Titan model config in the script directory (i.e: https://github.com/pytorch/torchtitan/blob/main/torchtitan/models/llama3/train_configs/debug_model.toml)

### 1. Create your SLURM job
Configure parameters for your cluster:
- num_nodes: Number of nodes to allocate (default: 2)
- gpus_per_node: Number of GPUs per node (default: 8)
- mesh_name: Name for the mesh (default: "mesh0")
- time_limit: Maximum job duration (default: "06:00:00")

In [1]:
# (c) Meta Platforms, Inc. and affiliates. Confidential and proprietary.

import logging
from monarch.job import SlurmJob

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(name)s %(asctime)s %(levelname)s %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
    force=True,
)
logger: logging.Logger = logging.getLogger(__name__)

# Configure job parameters
num_nodes = 2  # assign for your system
gpus_per_node = 4  # adjust for your hardware
mesh_name = "mesh0"

# Create a SLURM job with N nodes
slurm_job = SlurmJob(
    meshes={mesh_name: num_nodes},
    job_name="monarch_example",
    gpus_per_node=gpus_per_node,
    time_limit="06:00:00",
)

### 2. Define your Titan and cluster parameters

In [2]:
# (c) Meta Platforms, Inc. and affiliates. Confidential and proprietary.

from torchtitan.train import Trainer
from torchtitan.config import ConfigManager, JobConfig
from monarch.actor import Actor, current_rank, endpoint
from torchtitan.tools.logging import init_logger, logger
import torch
from dataclasses import dataclass
import os
from monarch.utils import setup_env_for_distributed


@dataclass
class RunParams:
    """
        Parameters for your cluster and training job, adjust as needed
    """
    training_steps: int = 50
    model_config = "debug_model.toml"
    dataset = "c4"
    num_nodes = num_nodes
    gpus_per_node = gpus_per_node


class TrainerActor(Actor):
    """
        A simple wrapper class with executes a TorchTitan trainer in a Monarch actor
    """
    def __init__(self, job_config: JobConfig) -> None:
        self.job_config = job_config
        rank = current_rank().rank
        self.uid = f"[trainer_{rank}]"

    @endpoint
    async def start_training(self) -> None:
        init_logger()
        trainer: Trainer | None = None

        try:
            trainer = Trainer(self.job_config)
            logger.info(f"{self.uid} initialized successfully and starting training")
            trainer.train()
        except Exception:
            if trainer:
                trainer.close()
            raise
        else:
            trainer.close()
        finally:
            torch.distributed.destroy_process_group()
            logger.info(f"{self.uid} trainer cleaned up")

def make_job_config() -> JobConfig:
    """
        Create a job config which is digested by TorchTitan, sourced from RunParams
    """
    data_parallel_shard_degree = RunParams.num_nodes * RunParams.gpus_per_node
    output_path = "./outputs"

    script_dir = globals()['_dh'][0]
    default_args = [
        "--job.config_file",
        os.path.join(script_dir, RunParams.model_config),
        "--model.tokenizer_path",
        os.path.join(script_dir, "tokenizer"),
        "--comm.trace_buf_size",
        "0",
        "--metrics.log_freq",
        "1",
        "--parallelism.data_parallel_shard_degree",
        str(data_parallel_shard_degree),
        "--activation_checkpoint.mode",
        "full",
        "--comm.train_timeout_seconds",
        "60",
        "--training.steps",
        str(RunParams.training_steps),
        "--training.dataset",
        RunParams.dataset,
        "--job.dump_folder",
        output_path,
        "--metrics.enable_tensorboard",
    ]

    config_manager = ConfigManager()
    job_config = config_manager.parse_args(default_args)

    return job_config

### 3. Execute your training job
You can make adjustments and run this on the existing SLURM allocations as many times as you would like!

In [3]:
# (c) Meta Platforms, Inc. and affiliates. Confidential and proprietary.

async def main():
    job_config = make_job_config()

    try:
        # 1. Get job state and create process mesh
        job_state = slurm_job.state()
        proc_mesh = job_state.mesh0.spawn_procs({"gpus": RunParams.gpus_per_node})
        
        # 2. Configure remote logging behavior
        await proc_mesh.logging_option(
            stream_to_client=True,
            # aggregate_window_sec=None  # Uncomment to disable log batching
        )
        
        # 3. Setup environment for torch.distributed
        await setup_env_for_distributed(proc_mesh)
        
        # 4. Spawn TrainerActor on each GPU
        trainer = proc_mesh.spawn("trainer_actor", TrainerActor, job_config)
        
        # 5. Execute the training job
        await trainer.start_training.call()
        
        logger.info("Training completed successfully!")
        
    except Exception as e:
        logger.error(f"Training workflow failed: {e}")


if __name__ == "__main__":
    await main()

root 2025-11-18 22:37:27 WARNING tokenizer_path is deprecated, use model.hf_assets_path instead. Setting hf_assets_path to tokenizer_path temporarily.
Found cached job at path: .monarch/job_state.pkl
SLURM job 9231 not found in queue
Cached job cannot run this spec, removing cache
Cancelled SLURM job 9231
Applying current job
Submitting SLURM job with 2 nodes
SLURM job 9233 submitted. Logs will be written to: /home/mreso/monarch/examples/slurm_9233_monarch_example_2659661.out
Saving job to cache at .monarch/job_state.pkl
Job has started, connecting to current state


SLURM job 9233 is running on 2 nodes: ['slurm-compute-node-090', 'slurm-compute-node-098']


>>> Aggregated Logs (2025-11-18 22:37:38) >>>
[1 similar log lines] [1] [titan] 2025-11-18 22:37:49,545 - root - INFO - Starting job: Llama 3 debug training
<<< Aggregated Logs (2025-11-18 22:37:49) <<<



>>> Aggregated Logs (2025-11-18 22:37:38) >>>
[8 similar log lines] [3] /home/mreso/miniforge3/envs/monarch/lib/python3.12/site-packages/torch/distributed/device_mesh.py:604: UserWarning: Slicing a flattened dim from root mesh will be deprecated in PT 2.11. Users need to bookkeep the flattened mesh directly. 
[8 similar log lines] [3]   sliced_mesh_layout = self._get_slice_mesh_layout(mesh_dim_names)
<<< Aggregated Logs (2025-11-18 22:37:52) <<<



>>> Aggregated Logs (2025-11-18 22:37:49) >>>
[7 similar log lines] [3] [titan] 2025-11-18 22:37:49,545 - root - INFO - Starting job: Llama 3 debug training
[8 similar log lines] [3] [titan] 2025-11-18 22:37:50,244 - root - INFO - Building 1-D device mesh with ['dp_shard'], [8]
[8 similar log lines] [3] [titan] 2025-11-18 22:37:50,247 - root - INFO - [GC] Initial GC collection took 0.00 seconds
<<< Aggregated Logs (2025-11-18 22:37:52) <<<

>>> Aggregated Logs (2025-11-18 22:37:52) >>>
[1 similar log lines] [5] [titan] 2025-11-18 22:37:56,210 - root - INFO - Loading tokenizer from tokenizer.json
<<< Aggregated Logs (2025-11-18 22:37:56) <<<

>>> Aggregated Logs (2025-11-18 22:37:56) >>>
[7 similar log lines] [6] [titan] 2025-11-18 22:37:56,210 - root - INFO - Loading tokenizer from tokenizer.json
[8 similar log lines] [6] [titan] 2025-11-18 22:37:56,232 - root - INFO - Preparing c4 dataset from allenai/c4
<<< Aggregated Logs (2025-11-18 22:37:59) <<<



...

>>> Aggregated Logs (2025-11-18 22:37:59) >>>
[8 similar log lines] [4] [titan] 2025-11-18 22:38:00,340 - root - INFO - Building llama3 debugmodel with TransformerModelArgs(_enforced='This field is used to enforce all fields have defaults.', dim=256, n_layers=6, n_heads=16, n_kv_heads=None, vocab_size=2048, multiple_of=256, ffn_dim_multiplier=None, norm_eps=1e-05, rope_theta=500000, rope_scaling_args=RoPEScalingArgs(scaling_factor=8.0, low_freq_factor=1.0, high_freq_factor=4.0, original_max_position_embeddings=8192), max_seq_len=2048, depth_init=True, use_flex_attn=False, attn_mask_type='causal', eos_id=0)
[8 similar log lines] [4] [titan] 2025-11-18 22:38:00,349 - root - INFO - CUDA capacity: NVIDIA GB200 with 184.00GiB memory
[8 similar log lines] [5] [titan] 2025-11-18 22:38:00,408 - root - INFO - Model llama3 debugmodel size: 6,163,712 total parameters
[8 similar log lines] [5] [titan] 2025-11-18 22:38:00,408 - root - INFO - Applied full activation checkpointing to the model
[8 sim

>>> Aggregated Logs (2025-11-18 22:38:02) >>>
[1 similar log lines] [0] /home/mreso/miniforge3/envs/monarch/lib/python3.12/site-packages/torch/distributed/device_mesh.py:604: UserWarning: Slicing a flattened dim from root mesh will be deprecated in PT 2.11. Users need to bookkeep the flattened mesh directly. 
[1 similar log lines] [0]   sliced_mesh_layout = self._get_slice_mesh_layout(mesh_dim_names)
<<< Aggregated Logs (2025-11-18 22:38:05) <<<

>>> Aggregated Logs (2025-11-18 22:38:05) >>>
[7 similar log lines] [3] /home/mreso/miniforge3/envs/monarch/lib/python3.12/site-packages/torch/distributed/device_mesh.py:604: UserWarning: Slicing a flattened dim from root mesh will be deprecated in PT 2.11. Users need to bookkeep the flattened mesh directly. 
[7 similar log lines] [3]   sliced_mesh_layout = self._get_slice_mesh_layout(mesh_dim_names)
<<< Aggregated Logs (2025-11-18 22:38:08) <<<



>>> Aggregated Logs (2025-11-18 22:38:02) >>>
[272 similar log lines] [3] [titan] 2025-11-18 22:38:05,821 - root - INFO - step:  1  loss:  8.1109  grad_norm:  1.5071  memory:  0.68GiB(0.37%)  tps: 3,482  tflops: 0.25  mfu: 0.01%
[8 similar log lines] [5] [titan] 2025-11-18 22:38:05,827 - root - INFO - Synchronizing and adjusting timeout for all ProcessGroups to 0:01:00
<<< Aggregated Logs (2025-11-18 22:38:08) <<<

>>> Aggregated Logs (2025-11-18 22:38:08) >>>
[128 similar log lines] [7] [titan] 2025-11-18 22:38:08,832 - root - INFO - step: 35  loss:  2.8697  grad_norm:  0.1972  memory:  0.69GiB(0.37%)  tps: 188,801  tflops: 13.52  mfu: 0.60%
[8 similar log lines] [7] [titan] 2025-11-18 22:38:10,140 - root - INFO - [GC] Performing periodic GC collection took 0.05 seconds
[7 similar log lines] [2] [titan] 2025-11-18 22:38:10,236 - root - INFO - Training completed
[1 similar log lines] [0] [titan] 2025-11-18 22:38:10,236 - root - INFO - Sleeping 2 seconds for other ranks to complete
[7 s

root 2025-11-18 22:38:12 INFO Training completed successfully!


>>> Aggregated Logs (2025-11-18 22:38:11) >>>
[1 similar log lines] [0] [titan] 2025-11-18 22:38:12,237 - root - INFO - Training completed
[1 similar log lines] [0] [titan] 2025-11-18 22:38:12,287 - root - INFO - [trainer_0] trainer cleaned up
<<< Aggregated Logs (2025-11-18 22:38:13) <<<



### 4. Cleanup the SLURM job
Once you're done experimenting, free up the allocation

In [ ]:
# (c) Meta Platforms, Inc. and affiliates. Confidential and proprietary.

# Cancel the SLURM job, releasing all reserved nodes back to the cluster
slurm_job.kill()
logger.info("Job terminated successfully")

Cancelled SLURM job 9233
root 2025-11-18 22:38:14 INFO Job terminated successfully


[-]E1118 22:38:45.979907 2659661 hyperactor/src/channel/net.rs:889] error_msg:session tcp:172.27.60.155:22222.9291070674861734285: failed to receive ack within timeout 30 secs; link is currently broken
[-]E1118 22:38:45.980013 2659661 hyperactor/src/mailbox.rs:344] name:undelivered_message_attempt, sender:tcp:172.27.55.7:43973,mesh_root_client_proc,client[0], dest:tcp:172.27.60.155:22222,service,agent[0][15904432331645146645<hyperactor_mesh::resource::GetState<hyperactor_mesh::v1::host_mesh::mesh_agent::ProcState>>], error:broken link: failed to enqueue in MailboxClient when processing buffer: channel closed, return_handle:tcp:172.27.55.7:43973,mesh_root_client_proc,client[0]<hyperactor::mailbox::undeliverable::Undeliverable<hyperactor::mailbox::MessageEnvelope>>
[-]E1118 22:38:45.980025 2659661 hyperactor/src/mailbox.rs:344] name:undelivered_message_attempt, sender:tcp:172.27.55.7:43973,mesh_root_client_proc,client[0], dest:tcp:172.27.60.155:22222,service,agent[0][15904432331645146645